In [1]:
!git clone https://github.com/NavairaRehman/Neural3DMM.git
%cd Neural3DMM

Cloning into 'Neural3DMM'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 115 (delta 61), reused 101 (delta 57), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 1.30 MiB | 2.32 MiB/s, done.
Resolving deltas: 100% (61/61), done.
/content/Neural3DMM


In [2]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 707.0/707.0 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully uninstalled jupyter-client-6.1.12
  Attempting unins

In [3]:
!pip install git+https://github.com/MPI-IS/mesh.git

  Cloning https://github.com/MPI-IS/mesh.git to /tmp/pip-req-build-indkni4m
  Running command git clone --filter=blob:none --quiet https://github.com/MPI-IS/mesh.git /tmp/pip-req-build-indkni4m
  Resolved https://github.com/MPI-IS/mesh.git to commit 49e70425cf373ec5269917012bda2944215c5ccd
  Preparing metadata (setup.py) ... done
  Created wheel for psbody-mesh: filename=psbody_mesh-0.4-cp311-cp311-linux_x86_64.whl size=2321656 sha256=caf7376f6817e9208c8e758d7a2426e490520fdc162a877a205c0313a0796d56
  Stored in directory: /tmp/pip-ephem-wheel-cache-vnds_qbm/wheels/f9/3c/ac/b3d1c84ec8972b9c7d1bf82a23432a142fa6bb4fba62c6c4d8
Successfully built psbody-mesh


## Prepping for COMA

In [4]:
%cd Neural3DMM
!mkdir root
%cd root
!mkdir COMA
%cd COMA
!mkdir preprocessed
!mkdir template
%cd preprocessed

[Errno 2] No such file or directory: 'Neural3DMM'
/content/Neural3DMM
/content/Neural3DMM/root
/content/Neural3DMM/root/COMA
/content/Neural3DMM/root/COMA/preprocessed


In [5]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#copy train.npy and test.npy from MyDrive to /content/Neural3DMM/root/COMA/preprocessed
import shutil
shutil.copy('/content/drive/MyDrive/train.npy', '/content/Neural3DMM/root/COMA/preprocessed')
shutil.copy('/content/drive/MyDrive/test.npy', '/content/Neural3DMM/root/COMA/preprocessed')

'/content/Neural3DMM/root/COMA/preprocessed/test.npy'

In [7]:
%cd ..
%cd template
!mkdir COMA_downsample
%cd COMA_downsample

/content/Neural3DMM/root/COMA
/content/Neural3DMM/root/COMA/template
/content/Neural3DMM/root/COMA/template/COMA_downsample


## Remaining

In [ ]:
!unzip /content/root_data.zip -d /content/root_data

Archive:  /content/root_data.zip
   creating: /content/root_data/root_data/
   creating: /content/root_data/root_data/Facial_Norms/
   creating: /content/root_data/root_data/Facial_Norms/preprocessed/
  inflating: /content/root_data/root_data/Facial_Norms/preprocessed/mean.npy  
  inflating: /content/root_data/root_data/Facial_Norms/preprocessed/std.npy  
  inflating: /content/root_data/root_data/Facial_Norms/preprocessed/test.npy  
  inflating: /content/root_data/root_data/Facial_Norms/preprocessed/train.npy  
   creating: /content/root_data/root_data/Facial_Norms/results/
   creating: /content/root_data/root_data/Facial_Norms/results/spirals_autoencoder/
   creating: /content/root_data/root_data/Facial_Norms/results/spirals_autoencoder/latent_16/
   creating: /content/root_data/root_data/Facial_Norms/results/spirals_autoencoder/latent_16/checkpoints/
   creating: /content/root_data/root_data/Facial_Norms/results/spirals_autoencoder/latent_16/predictions/
   creating: /content/root_da

In [2]:
%cd /content/Neural3DMM

/content/Neural3DMM


In [3]:
import numpy as np
import json
import os
import copy
import pickle

import mesh_sampling
import trimesh
from shape_data import ShapeData

from autoencoder_dataset import autoencoder_dataset
from torch.utils.data import DataLoader

from spiral_utils import get_adj_trigs, generate_spirals
from models import SpiralAutoencoder
from train_funcs import train_autoencoder_dataloader
from test_funcs import test_autoencoder_dataloader


import torch
from tensorboardX import SummaryWriter

from sklearn.metrics.pairwise import euclidean_distances
meshpackage = 'mpi-mesh' # 'mpi-mesh', 'trimesh'
root_dir = '/content/Neural3DMM/root'

dataset = 'COMA'
name = ''

GPU = True
device_idx = 0
torch.cuda.get_device_name(device_idx)

'Tesla T4'

In [4]:
args = {}

generative_model = 'autoencoder'
downsample_method = 'COMA_downsample' # choose'COMA_downsample' or 'meshlab_downsample'


# below are the arguments for the DFAUST run
reference_mesh_file = os.path.join(root_dir, dataset, 'template', 'template.obj')
downsample_directory = os.path.join(root_dir, dataset,'template', downsample_method)
ds_factors = [4, 4, 4, 4]
step_sizes = [2, 2, 1, 1, 1]
filter_sizes_enc = [[3, 16, 32, 64, 128],[[],[],[],[],[]]]
filter_sizes_dec = [[128, 64, 32, 32, 16],[[],[],[],[],3]]
dilation_flag = True
if dilation_flag:
    dilation=[2, 2, 1, 1, 1]
else:
    dilation = None
reference_points = [[414]]  # [[3567,4051,4597]] used for COMA with 3 disconnected components

args = {'generative_model': generative_model,
        'name': name, 'data': os.path.join(root_dir, dataset, 'preprocessed',name),
        'results_folder':  os.path.join(root_dir, dataset,'results/spirals_'+ generative_model),
        'reference_mesh_file':reference_mesh_file, 'downsample_directory': downsample_directory,
        'checkpoint_file': 'checkpoint',
        'seed':2, 'loss':'l1',
        'batch_size':16, 'num_epochs':300, 'eval_frequency':200, 'num_workers': 4,
        'filter_sizes_enc': filter_sizes_enc, 'filter_sizes_dec': filter_sizes_dec,
        'nz':16,
        'ds_factors': ds_factors, 'step_sizes' : step_sizes, 'dilation': dilation,

        'lr':1e-3,
        'regularization': 5e-5,
        'scheduler': True, 'decay_rate': 0.99,'decay_steps':1,
        'resume': False,

        'mode':'train', 'shuffle': True, 'nVal': 100, 'normalization': True}

args['results_folder'] = os.path.join(args['results_folder'],'latent_'+str(args['nz']))

if not os.path.exists(os.path.join(args['results_folder'])):
    os.makedirs(os.path.join(args['results_folder']))

summary_path = os.path.join(args['results_folder'],'summaries',args['name'])
if not os.path.exists(summary_path):
    os.makedirs(summary_path)

checkpoint_path = os.path.join(args['results_folder'],'checkpoints', args['name'])
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)

samples_path = os.path.join(args['results_folder'],'samples', args['name'])
if not os.path.exists(samples_path):
    os.makedirs(samples_path)

prediction_path = os.path.join(args['results_folder'],'predictions', args['name'])
if not os.path.exists(prediction_path):
    os.makedirs(prediction_path)

if not os.path.exists(downsample_directory):
    os.makedirs(downsample_directory)

In [5]:
from psbody.mesh import Mesh
import mesh_sampling

np.random.seed(args['seed'])
print("Loading data .. ")
if not os.path.exists(args['data']+'/mean.npy') or not os.path.exists(args['data']+'/std.npy'):
    shapedata =  ShapeData(nVal=args['nVal'],
                          train_file=args['data']+'/train.npy',
                          test_file=args['data']+'/test.npy',
                          reference_mesh_file=args['reference_mesh_file'],
                          normalization = args['normalization'],
                          meshpackage = meshpackage, load_flag = True)
    np.save(args['data']+'/mean.npy', shapedata.mean)
    np.save(args['data']+'/std.npy', shapedata.std)
else:
    shapedata = ShapeData(nVal=args['nVal'],
                         train_file=args['data']+'/train.npy',
                         test_file=args['data']+'/test.npy',
                         reference_mesh_file=args['reference_mesh_file'],
                         normalization = args['normalization'],
                         meshpackage = meshpackage, load_flag = False)
    shapedata.mean = np.load(args['data']+'/mean.npy')
    shapedata.std = np.load(args['data']+'/std.npy')
    shapedata.n_vertex = shapedata.mean.shape[0]
    shapedata.n_features = shapedata.mean.shape[1]

if not os.path.exists(os.path.join(args['downsample_directory'],'downsampling_matrices.pkl')):
    if shapedata.meshpackage == 'trimesh':
        raise NotImplementedError('Rerun with mpi-mesh as meshpackage')
    print("Generating Transform Matrices ..")
    if downsample_method == 'COMA_downsample':
        M,A,D,U,F = mesh_sampling.generate_transform_matrices(shapedata.reference_mesh, args['ds_factors'])
    with open(os.path.join(args['downsample_directory'],'downsampling_matrices.pkl'), 'wb') as fp:
        M_verts_faces = [(M[i].v, M[i].f) for i in range(len(M))]
        pickle.dump({'M_verts_faces':M_verts_faces,'A':A,'D':D,'U':U,'F':F}, fp)
else:
    print("Loading Transform Matrices ..")
    with open(os.path.join(args['downsample_directory'],'downsampling_matrices.pkl'), 'rb') as fp:
        #downsampling_matrices = pickle.load(fp,encoding = 'latin1')
        downsampling_matrices = pickle.load(fp)

    M_verts_faces = downsampling_matrices['M_verts_faces']
    if shapedata.meshpackage == 'mpi-mesh':
        M = [Mesh(v=M_verts_faces[i][0], f=M_verts_faces[i][1]) for i in range(len(M_verts_faces))]
    elif shapedata.meshpackage == 'trimesh':
        M = [trimesh.base.Trimesh(vertices=M_verts_faces[i][0], faces=M_verts_faces[i][1], process = False) for i in range(len(M_verts_faces))]
    A = downsampling_matrices['A']
    D = downsampling_matrices['D']
    U = downsampling_matrices['U']
    F = downsampling_matrices['F']

# Needs also an extra check to enforce points to belong to different disconnected component at each hierarchy level
print("Calculating reference points for downsampled versions..")
for i in range(len(args['ds_factors'])):
    if shapedata.meshpackage == 'mpi-mesh':
        dist = euclidean_distances(M[i+1].v, M[0].v[reference_points[0]])
    elif shapedata.meshpackage == 'trimesh':
        dist = euclidean_distances(M[i+1].vertices, M[0].vertices[reference_points[0]])
    reference_points.append(np.argmin(dist,axis=0).tolist())



Loading data .. 
Loading Transform Matrices ..
Calculating reference points for downsampled versions..


In [7]:
if shapedata.meshpackage == 'mpi-mesh':
    sizes = [x.v.shape[0] for x in M]
elif shapedata.meshpackage == 'trimesh':
    sizes = [x.vertices.shape[0] for x in M]
Adj, Trigs = get_adj_trigs(A, F, shapedata.reference_mesh, meshpackage = shapedata.meshpackage)

spirals_np, spiral_sizes,spirals = generate_spirals(args['step_sizes'],
                                                    M, Adj, Trigs,
                                                    reference_points = reference_points,
                                                    dilation = args['dilation'], random = False,
                                                    meshpackage = shapedata.meshpackage,
                                                    counter_clockwise = True)

bU = []
bD = []
for i in range(len(D)):
    d = np.zeros((1,D[i].shape[0]+1,D[i].shape[1]+1))
    u = np.zeros((1,U[i].shape[0]+1,U[i].shape[1]+1))
    d[0,:-1,:-1] = D[i].todense()
    u[0,:-1,:-1] = U[i].todense()
    d[0,-1,-1] = 1
    u[0,-1,-1] = 1
    bD.append(d)
    bU.append(u)


spiral generation for hierarchy 0 (5023 vertices) finished
spiral generation for hierarchy 1 (1256 vertices) finished
spiral generation for hierarchy 2 (314 vertices) finished
spiral generation for hierarchy 3 (79 vertices) finished
spiral generation for hierarchy 4 (20 vertices) finished
spiral sizes for hierarchy 0:  16
spiral sizes for hierarchy 1:  16
spiral sizes for hierarchy 2:  10
spiral sizes for hierarchy 3:  10
spiral sizes for hierarchy 4:  8


In [8]:
torch.manual_seed(args['seed'])

if GPU:
    device = torch.device("cuda:"+str(device_idx) if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

tspirals = [torch.from_numpy(s).long().to(device) for s in spirals_np]
tD = [torch.from_numpy(s).float().to(device) for s in bD]
tU = [torch.from_numpy(s).float().to(device) for s in bU]

cuda:0


In [9]:
!python data_generation.py --root_dir=/content/Neural3DMM/root --dataset=COMA --num_valid=500

100% 17915/17915 [00:09<00:00, 1910.60it/s]


In [ ]:
#download the root folder to run locally
#zip root using gz
import tarfile

def compress_directory_with_gz(directory_path, output_path):
    with tarfile.open(output_path, "w:gz") as tar:
        tar.add(directory_path, arcname=".")

# Usage example:
directory_to_compress = "/content/Neural3DMM/root"
output_gz_file = "/content/archive.tar.gz"
compress_directory_with_gz(directory_to_compress, output_gz_file)


In [2]:
import os

def convert_bytes(size):
    """Convert bytes to KB, MB, or GB."""
    for unit in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024:
            return f"{size:.2f} {unit}"
        size /= 1024

# Replace 'your_file_path' with the actual path to your file
file_path = '/content/root.zip'

try:
    # Get the file size in bytes
    file_size = os.path.getsize(file_path)
    # Convert the file size to a human-readable format
    readable_size = convert_bytes(file_size)
    print(f"The size of the file is: {readable_size}")
except FileNotFoundError:
    print("File not found. Please check the file path.")
except OSError:
    print("An error occurred while accessing the file.")


The size of the file is: 2.11 GB


In [3]:
from google.colab import files

# Replace 'your_file.txt' with the path to your file
files.download('/content/root.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# Building model, optimizer, and loss function

dataset_train = autoencoder_dataset(root_dir = args['data'], points_dataset = 'train',
                                           shapedata = shapedata,
                                           normalization = args['normalization'])

dataloader_train = DataLoader(dataset_train, batch_size=args['batch_size'],\
                                     shuffle = args['shuffle'], num_workers = args['num_workers'])

dataset_val = autoencoder_dataset(root_dir = args['data'], points_dataset = 'val',
                                         shapedata = shapedata,
                                         normalization = args['normalization'])

dataloader_val = DataLoader(dataset_val, batch_size=args['batch_size'],\
                                     shuffle = False, num_workers = args['num_workers'])


dataset_test = autoencoder_dataset(root_dir = args['data'], points_dataset = 'test',
                                          shapedata = shapedata,
                                          normalization = args['normalization'])

dataloader_test = DataLoader(dataset_test, batch_size=args['batch_size'],\
                                     shuffle = False, num_workers = args['num_workers'])



if 'autoencoder' in args['generative_model']:
        model = SpiralAutoencoder(filters_enc = args['filter_sizes_enc'],
                                  filters_dec = args['filter_sizes_dec'],
                                  latent_size=args['nz'],
                                  sizes=sizes,
                                  spiral_sizes=spiral_sizes,
                                  spirals=tspirals,
                                  D=tD, U=tU,device=device).to(device)


optim = torch.optim.Adam(model.parameters(),lr=args['lr'],weight_decay=args['regularization'])
if args['scheduler']:
    scheduler=torch.optim.lr_scheduler.StepLR(optim, args['decay_steps'],gamma=args['decay_rate'])
else:
    scheduler = None

if args['loss']=='l1':
    def loss_l1(outputs, targets):
        L = torch.abs(outputs - targets).mean()
        return L
    loss_fn = loss_l1


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [11]:
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of parameters is: {}".format(params))
print(model)
# print(M[4].v.shape)

Total number of parameters is: 328211
SpiralAutoencoder(
  (encoder): SpiralEncoder(
    (conv): ModuleList(
      (0): SpiralConv(
        (conv): Linear(in_features=48, out_features=16, bias=True)
        (activation): ELU(alpha=1.0)
      )
      (1): SpiralConv(
        (conv): Linear(in_features=256, out_features=32, bias=True)
        (activation): ELU(alpha=1.0)
      )
      (2): SpiralConv(
        (conv): Linear(in_features=320, out_features=64, bias=True)
        (activation): ELU(alpha=1.0)
      )
      (3): SpiralConv(
        (conv): Linear(in_features=640, out_features=128, bias=True)
        (activation): ELU(alpha=1.0)
      )
    )
    (fc_latent_enc): Linear(in_features=2688, out_features=16, bias=True)
  )
  (decoder): SpiralDecoder(
    (fc_latent_dec): Linear(in_features=16, out_features=2688, bias=True)
    (dconv): ModuleList(
      (0): SpiralConv(
        (conv): Linear(in_features=1280, out_features=64, bias=True)
        (activation): ELU(alpha=1.0)
      )

In [14]:
if args['mode'] == 'train':
    writer = SummaryWriter(summary_path)
    with open(os.path.join(args['results_folder'], 'checkpoints', args['name'] + '_params.json'), 'w') as fp:
        saveparams = copy.deepcopy(args)
        json.dump(saveparams, fp)

    if args['resume']:
        print('loading checkpoint from file %s' % (os.path.join(checkpoint_path, args['checkpoint_file'])))
        checkpoint_dict = torch.load(os.path.join(checkpoint_path, args['checkpoint_file'] + '.pth.tar'), map_location=device)
        start_epoch = checkpoint_dict['epoch'] + 1
        model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
        optim.load_state_dict(checkpoint_dict['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint_dict['scheduler_state_dict'])
        print('Resuming from epoch %s' % (str(start_epoch)))
    else:
        start_epoch = 0

    if args['generative_model'] == 'autoencoder':
        train_autoencoder_dataloader(dataloader_train, dataloader_val,
                                      device, model, optim, loss_fn,
                                      start_epoch=start_epoch,
                                      n_epochs=args['num_epochs'],
                                      eval_freq=args['eval_frequency'],
                                      scheduler=scheduler,
                                      writer=writer,
                                      save_recons=True,
                                      shapedata=shapedata,
                                      metadata_dir=checkpoint_path,
                                      samples_dir=samples_path,
                                      checkpoint_path=args['checkpoint_file'])


  0%|          | 0/1120 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
100%|██████████| 32/32 [00:01<00:00, 26.28it/s]


epoch 0 | tr 0.3037162799368416 | val 0.181982364654541


NameError: name 'tx' is not defined

In [ ]:
if args['mode'] == 'test':
    print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar')))
    checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
    model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])

    predictions, norm_l1_loss, l2_loss = test_autoencoder_dataloader(device, model, dataloader_test,
                                                                     shapedata, mm_constant = 1000)
    np.save(os.path.join(prediction_path,'predictions'), predictions)

    print('autoencoder: normalized loss', norm_l1_loss)

    print('autoencoder: euclidean distance in mm=', l2_loss)